In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    %pip uninstall -y plotnine
    !git clone https://github.com/bzhanglab/CoPheeMap.git
    %cd CoPheeMap
    %pip install -q -r requirements.txt

In [3]:
import pandas as pd
import re
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
import math
import xlrd
from collections import Counter
import os

In [4]:
kinases_id=pd.read_excel('KSA/kinase_table.xlsx')
kinases_id.index=kinases_id.Name
K_S_GPS=pd.read_csv('KSA/KSA_gold_standard.csv',index_col=0)

In [5]:
#Generate PSSM matrix from gold standard KSAs 
#Warning: Long running time
ks=list(set(K_S_GPS['Gene name']))
for k in ks:
    df=pd.DataFrame()
    tmp=K_S_GPS[K_S_GPS['Gene name']==k]
    seqs=tmp.seq_15.to_list()
    seqs=list(set(seqs))
    if len(seqs) >= 5:
        aa_lib = ["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
        for a in aa_lib:
            num1=[]
            for i in range(15):
                aa=[s[i] for s in seqs]
                c=Counter(aa)
                num1.append(c[a])
            df[a]=num1
        df=df.transpose()
        df.to_csv('CoPheeKSA/PSSM/Kinase_5+sub/'+k+'_pssm.csv')

In [9]:
lst=os.listdir('CoPheeKSA/PSSM/Kinase_5+sub/')
os.mkdir('CoPheeKSA/PSSM/removed_test')
for i in range(10):
    os.mkdir('CoPheeKSA/PSSM/removed_test/'+str(i))
ks=[i.split('_')[0] for i in lst]
#Generate PSSM matrix from gold standard KSAs which already removed test data
for j in range(10):
    test=pd.read_csv('CoPheeKSA/test/test_'+str(j)+'.csv',index_col=0)
    
    for k in ks:
        test_tmp=test[test.kinases==k]
        test_sites=test_tmp.sites.tolist()
        test_seqs=[s.split('|')[3] for s in test_sites]
        
        df=pd.DataFrame()
        tmp=K_S_GPS[K_S_GPS['Gene name']==k]
        seqs=tmp.seq_15.to_list()
        seqs=list(set(seqs).difference(set(test_seqs)))
        
        if len(seqs) >= 5:
            aa_lib = ["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]
            for a in aa_lib:
                num1=[]
                for i in range(15):
                    aa=[s[i] for s in seqs]
                    c=Counter(aa)
                    num1.append(c[a])
                df[a]=num1
            df=df.transpose()
            df.to_csv('CoPheeKSA/PSSM/removed_test/'+str(j)+'/'+k+'_pssm.csv')